# Обеспечение качества и безопасности данных в системах машинного зрения

## Кейс:

Вы работаете инженером машинного зрения в сфере БАС. Руководство Вашего подразделения попросило передать мини-датасет изображений и сопутствущую телеметрию субподрядчику для выполнения заказанных работ. Вы знаете что:
- для работы им нужны только некторые данные в телеметрии — а именно данные по динамике полета (высота, углы…);
- у них система «заточена» под общий формат (стандарт телеметрии для БАС данного типа дататсетов): строго определенное число столбцов и навазния столбцов, при этом ячейки не должны быть пустыми (или содержать значения, интерпретируемые как пустые — пустые строки, например). Ваш файл телеметри полностью соответствует стандарту;
- помните случай в прошлом году, когда утечка координат и времени полета из файла телемеитрии БАС-опрератора «Компании X» привела ее к серьезным убыткам;
- не знаете никого в организации субподрядчика и в первый раз с ними работате…

Проявляя разумную осмотрительность на этапе передачи данных, Вы решаете (предварительно договрившись с субподрядчиком использовать защищенный протокол передачи данных, например SSH):
1. Зашифровать столбцы телеметрии с данными координат и времени полета.
2. Удалить все EXIF-метаданные изображений

## Для этого Вам надо:

1. Прочитать датасет в pandas и зашифровать соотвеотствующие данные, а также
2. Написать функцию, которая будет копировать изображения в другую директорию, но уже
без метаданных.

## Материал для выполнения:

__Архив, содержащий:__
1. Набор из нескольких изображений с метаданными «0001.jpg, 0002.jpg...»
2. Файл фрагмента телеметрии БАС «tele_part.xlsx»

## Работа над кейсом

### 1. Импорт необходимых библиотек

In [27]:
import pandas as pd
import hashlib as hl
import PIL.Image as im
import PIL.ExifTags as tags
import os
from exif import Image as exim

### 2. Шифрование (кэширование) данных телеметрии

__Загрузка данных в pandas__

In [28]:
file_name = 'telemetry.csv'

ds = pd.read_csv(file_name)
ds.head()

,image,date,time,lat,lon,alt,pitch,roll,azimuth,GPS_speed
0,DSC00003.JPG,2020/04/30,"04:47:07,00",61.521366,30.339928,18.3,0.5,0.0,140.0,0.00
1,DSC00004.JPG,2020/04/30,"04:47:07,00",61.521366,30.339928,18.3,0.5,0.0,140.0,0.00
2,DSC00005.JPG,2020/04/30,"04:47:09,00",61.521366,30.339930,18.4,0.6,-0.4,140.0,0.00
3,DSC00006.JPG,2020/04/30,"04:47:10,00",61.521366,30.339931,18.6,0.6,-0.5,140.0,0.00
4,DSC00007.JPG,2020/04/30,"05:00:10,00",61.503338,30.437277,295.6,0.7,1.8,97.0,81.36


__Функция хэширования__

* Используем функцию хэширования для значения ячеек столбца column и строки row
* Значение предварительно переводим в строчный формат, т.к. формат float, имеющийся в ячейках не будет кодироваться

In [29]:
def hash_data(column, row):
    return hl.sha256(str(row[column]).encode('utf-8')).hexdigest()

__Функция замены данных в наборе данных__

* Функция для построчного хеширования данных конкретного столбца реализуется с помощью метода apply в pandas
* Используем значение axis=1, что означаетпострочный обход столбца column
* Используем лямбда-функцию - значение номера строки в данном случае будет изменяться в соответствии с проходом метода apply.

In [30]:
def replace_data(column):
    ds[column] = ds.apply(lambda x: hash_data(column, x), axis=1)

__Кэшируем данные, не касающиеся динамики полета__

* Определяем параметры, которые необходимо оставить без изменений - согласно кейсу, характеристики, касающиеся полета.
* Остальные парметры хэшируем - итеративно хэшируем каждый столбец, за исключением определнных выше

In [31]:
fly_params = ['alt', 'pitch', 'roll', 'GPS_speed']
for col in ds.columns.tolist():
    if not col in fly_params:
        replace_data(col)
ds.head()

,image,date,time,lat,lon,alt,pitch,roll,azimuth,GPS_speed
0,6f85111652d5c6ea9d772b918efdb2a96aa043e220291c...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,56e6708a2f0be9a25986b1c8290b2fbec50fccf9539fa9...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,496b4ff3781ed341f5c3c43ad0bdc6b681833214dfb5b5...,18.3,0.5,0.0,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
1,1dd31e027c4ccc0ed487fbd94d6a21471796395bafa191...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,56e6708a2f0be9a25986b1c8290b2fbec50fccf9539fa9...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,496b4ff3781ed341f5c3c43ad0bdc6b681833214dfb5b5...,18.3,0.5,0.0,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
2,1b16eb81f45d75e58586785b4cbdc4d1123c022be92ee9...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,3b002224be6aceba1e9a7c713f7bae25edc96672e04de0...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,3df635a9c1c8720af19592df83ae438b2ddf8ab1b1a892...,18.4,0.6,-0.4,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
3,814ebc56bd745831bbfe037ea29812251e0e45c14798fe...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,b07c6a4a0e88130a58156a6829b72179372870a13cf532...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,ffc95fea2f89c53d5ccb697700832ccb60fb69f36fa708...,18.6,0.6,-0.5,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
4,80bf56b54251fc94d8803c8dae1a6a15281f8411a2f599...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,82ab96936882be49edb3fe7e5fef2c79380e58a1343ea9...,dd032c26f37c77f1ae378ae4dcbdae1b06a279c4771cac...,ce22e62b0cb7ed1e2f0ead97e220e06a59a2a140df7cec...,295.6,0.7,1.8,ee36dd09533bf8d7521539ccdb78a2f0e74bf582742e79...,81.36


__Сохраняем датасет в новый csv-файл__

* Добавляем в название исходного файла строку '_hash'

In [32]:
new_file_name = "".join(file_name.split('.')[:-1])+'_hash.csv'
ds.to_csv(new_file_name)

__Проверка создания файла__

In [33]:
hash_ds = pd.read_csv(new_file_name)
hash_ds.head()

,Unnamed: 0,image,date,time,lat,lon,alt,pitch,roll,azimuth,GPS_speed
0,0,6f85111652d5c6ea9d772b918efdb2a96aa043e220291c...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,56e6708a2f0be9a25986b1c8290b2fbec50fccf9539fa9...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,496b4ff3781ed341f5c3c43ad0bdc6b681833214dfb5b5...,18.3,0.5,0.0,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
1,1,1dd31e027c4ccc0ed487fbd94d6a21471796395bafa191...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,56e6708a2f0be9a25986b1c8290b2fbec50fccf9539fa9...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,496b4ff3781ed341f5c3c43ad0bdc6b681833214dfb5b5...,18.3,0.5,0.0,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
2,2,1b16eb81f45d75e58586785b4cbdc4d1123c022be92ee9...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,3b002224be6aceba1e9a7c713f7bae25edc96672e04de0...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,3df635a9c1c8720af19592df83ae438b2ddf8ab1b1a892...,18.4,0.6,-0.4,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
3,3,814ebc56bd745831bbfe037ea29812251e0e45c14798fe...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,b07c6a4a0e88130a58156a6829b72179372870a13cf532...,3090639c79c3272d7bf2e2946eb6958b4b2e26fbb4d7e9...,ffc95fea2f89c53d5ccb697700832ccb60fb69f36fa708...,18.6,0.6,-0.5,5d6ba45ee1d5e1697ad5f130677d44a7812ad273f13d6c...,0.00
4,4,80bf56b54251fc94d8803c8dae1a6a15281f8411a2f599...,5fc67389af8980a2d483c93d15f1a99509d1a960867825...,82ab96936882be49edb3fe7e5fef2c79380e58a1343ea9...,dd032c26f37c77f1ae378ae4dcbdae1b06a279c4771cac...,ce22e62b0cb7ed1e2f0ead97e220e06a59a2a140df7cec...,295.6,0.7,1.8,ee36dd09533bf8d7521539ccdb78a2f0e74bf582742e79...,81.36


### 3. Копирование изображений без метаданных

#### 3.1 Подготовка

__Сначала определяем все изображения из набора в список__

In [34]:
current_dir = os.path.abspath(os.curdir)
images = []
for f in os.scandir(current_dir):
    if f.is_file() and (
        f.path.split('.')[-1].lower() == 'jpg' or\
     f.path.split('.')[-1].lower() == 'jpeg'):
        images.append(f.path)
images

['/home/alex/bpla/venv/БАС/00_ДЗ/ДЗ_8/2411.JPG',
 '/home/alex/bpla/venv/БАС/00_ДЗ/ДЗ_8/338.JPG',
 '/home/alex/bpla/venv/БАС/00_ДЗ/ДЗ_8/343.JPG',
 '/home/alex/bpla/venv/БАС/00_ДЗ/ДЗ_8/1083.JPG']

__Проверим наличие метаданных на примере 1-го изображения из полученного списка__

* Загрузка изображения с помощью библиотеки Pillow

In [35]:
img = im.open(images[0])

* Выгружаем все имеющиеся метаданные изобржения в словарь

In [36]:
metadata = img._getexif()
metadata

{34853: {0: b'\x02\x00\x00\x00',
  1: 'N',
  2: (61.375687, 0.0, 0.0),
  3: 'E',
  4: (30.368956, 0.0, 0.0),
  5: 0,
  6: 291.6,
  18: 'WGS-84'},
 50341: b'PrintIM\x000300\x00\x00\x03\x00\x02\x00\x01\x00\x00\x00\x03\x00"\x00\x00\x00\x01\x01\x00\x00\x00\x00\t\x11\x00\x00\x10\'\x00\x00\x0b\x0f\x00\x00\x10\'\x00\x00\x97\x05\x00\x00\x10\'\x00\x00\xb0\x08\x00\x00\x10\'\x00\x00\x01\x1c\x00\x00\x10\'\x00\x00^\x02\x00\x00\x10\'\x00\x00\x8b\x00\x00\x00\x10\'\x00\x00\xcb\x03\x00\x00\x10\'\x00\x00\xe5\x1b\x00\x00\x10\'\x00\x00',
 296: 2,
 34665: 376,
 270: '                               ',
 271: 'SONY',
 272: 'ILCE-6000',
 305: 'ILCE-6000 v3.20',
 274: 3,
 306: '2020:04:30 08:48:36',
 531: 2,
 282: 350.0,
 283: 350.0,
 36864: b'0230',
 37121: b'\x01\x02\x03\x00',
 37122: 3.0,
 36867: '2020:04:30 08:48:36',
 36868: '2020:04:30 08:48:36',
 37379: 6.45078125,
 37380: 0.0,
 37381: 1.6953125,
 37383: 5,
 37384: 0,
 37385: 16,
 37386: 35.0,
 37510: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

* Чтобы понять, какая информация имеется в метаданных, используем сопоставление id тегов и их названий с помощью метода ExifTags.TAGS

In [37]:
exif_data = {
    tags.TAGS[k]: v
    for k, v in metadata.items()
    if k in tags.TAGS
}

exif_data

{'GPSInfo': {0: b'\x02\x00\x00\x00',
  1: 'N',
  2: (61.375687, 0.0, 0.0),
  3: 'E',
  4: (30.368956, 0.0, 0.0),
  5: 0,
  6: 291.6,
  18: 'WGS-84'},
 'PrintImageMatching': b'PrintIM\x000300\x00\x00\x03\x00\x02\x00\x01\x00\x00\x00\x03\x00"\x00\x00\x00\x01\x01\x00\x00\x00\x00\t\x11\x00\x00\x10\'\x00\x00\x0b\x0f\x00\x00\x10\'\x00\x00\x97\x05\x00\x00\x10\'\x00\x00\xb0\x08\x00\x00\x10\'\x00\x00\x01\x1c\x00\x00\x10\'\x00\x00^\x02\x00\x00\x10\'\x00\x00\x8b\x00\x00\x00\x10\'\x00\x00\xcb\x03\x00\x00\x10\'\x00\x00\xe5\x1b\x00\x00\x10\'\x00\x00',
 'ResolutionUnit': 2,
 'ExifOffset': 376,
 'ImageDescription': '                               ',
 'Make': 'SONY',
 'Model': 'ILCE-6000',
 'Software': 'ILCE-6000 v3.20',
 'Orientation': 3,
 'DateTime': '2020:04:30 08:48:36',
 'YCbCrPositioning': 2,
 'XResolution': 350.0,
 'YResolution': 350.0,
 'ExifVersion': b'0230',
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'CompressedBitsPerPixel': 3.0,
 'DateTimeOriginal': '2020:04:30 08:48:36',
 'DateTimeDi

#### 3.2 Удаление EXIF с помощью Pillow

__Функция создания нового изображения без метаданных__

In [38]:
def del_exif (input_img, savepth):
    img_name = input_img.split('/')[-1]
    img = im.open(input_img)

    data = list(img.getdata())
    image_without_exif = im.new(img.mode, img.size)
    image_without_exif.putdata(data)
    new_name = os.path.join(savepth, 'no_exif_' + img_name)
    image_without_exif.save(new_name)
    
    return f'exif cleared for {img_name}'

__Задаем директорию для сохранения изображений__

In [39]:
name_pth = 'no_exif'
if not os.path.exists(name_pth):
    os.mkdir(name_pth)
savepth = name_pth

__Применяем функцию удаления exif ко всем изображениям__

In [40]:
%%time

for i in images:
    del_exif(i, savepth)

CPU times: user 46 s, sys: 7.33 s, total: 53.3 s
Wall time: 53.7 s


__Проверяем наличие метаданных у новых изображений__

In [41]:
new_img_dir = os.path.join(os.path.abspath(os.curdir), savepth)
check_exif = []
for f in os.scandir(new_img_dir):
    temp = f.path
    img = im.open(f.path)
    check_exif.append(img._getexif())
check_exif

[None, None, None, None]

__Как виидим, функция получения метаданных вернула пустые словари для новых изображений__

#### 3.3 Удаление EXIF с помощью библиотеки EXIF

__Создадим словарь для загрузки объектов класса exif и название изображения. В нем ключами будут объекты класса exif, а значениями - названия__

In [42]:
exif_imgs = {}

__В цикле создаем экземпляры класса exif для всех изображений__

In [43]:
for i in images:
    with open(i, "rb") as img:
        img = exim(img)
    exif_imgs[img] = i.split('/')[-1]
exif_imgs

{<exif._image.Image at 0x7aef4cddd2e0>: '2411.JPG',
 <exif._image.Image at 0x7aef4c246120>: '338.JPG',
 <exif._image.Image at 0x7aef4c223740>: '343.JPG',
 <exif._image.Image at 0x7aef4c220f20>: '1083.JPG'}

__Посмотрим, имеются ли теги у объектов, и какие__

In [44]:
for k in exif_imgs.keys():
    print(k.has_exif)
    print(dir(k))

True
['<unknown EXIF tag 50341>', '_exif_ifd_pointer', '_gps_ifd_pointer', '_interoperability_ifd_Pointer', '_segments', 'brightness_value', 'color_space', 'components_configuration', 'compressed_bits_per_pixel', 'compression', 'contrast', 'custom_rendered', 'datetime', 'datetime_digitized', 'datetime_original', 'delete', 'delete_all', 'digital_zoom_ratio', 'exif_version', 'exposure_bias_value', 'exposure_mode', 'exposure_program', 'exposure_time', 'f_number', 'file_source', 'flash', 'flashpix_version', 'focal_length', 'focal_length_in_35mm_film', 'get', 'get_all', 'get_file', 'get_thumbnail', 'gps_altitude', 'gps_altitude_ref', 'gps_latitude', 'gps_latitude_ref', 'gps_longitude', 'gps_longitude_ref', 'gps_map_datum', 'gps_version_id', 'has_exif', 'image_description', 'jpeg_interchange_format', 'jpeg_interchange_format_length', 'lens_model', 'lens_specification', 'light_source', 'list_all', 'make', 'maker_note', 'max_aperture_value', 'metering_mode', 'model', 'orientation', 'photograph

__Мы видим значение True у всех объектов и список тегов. Далее удаляем все имеющиеся теги у объектов__

In [45]:
for k in exif_imgs.keys():
    k.delete_all()
    print(k.has_exif)

True


/home/alex/bpla/venv/lib/python3.12/site-packages/exif/_image.py:154: RuntimeWarning: could not delete tag <unknown EXIF tag 50341>
  warnings.warn("could not delete tag " + tag, RuntimeWarning)


True
True
True


__Как видно, не все теги удалилсь (даже появилось соответсвующее сообщение об ошибке). Однако это связано с тем, что объекты класса exif имеют в качестве тегов не только метаданные, но и служебные методы и функции. Посмотрим, каие теги остались в итоге.__

In [46]:
for k in exif_imgs.keys():
    print(dir(k))

['<unknown EXIF tag 50341>', '_exif_ifd_pointer', '_gps_ifd_pointer', '_segments', 'delete', 'delete_all', 'exif_version', 'get', 'get_all', 'get_file', 'get_thumbnail', 'has_exif', 'list_all']
['<unknown EXIF tag 50341>', '_exif_ifd_pointer', '_gps_ifd_pointer', '_segments', 'delete', 'delete_all', 'exif_version', 'get', 'get_all', 'get_file', 'get_thumbnail', 'has_exif', 'list_all']
['<unknown EXIF tag 50341>', '_exif_ifd_pointer', '_gps_ifd_pointer', '_segments', 'delete', 'delete_all', 'exif_version', 'get', 'get_all', 'get_file', 'get_thumbnail', 'has_exif', 'list_all']
['<unknown EXIF tag 50341>', '_exif_ifd_pointer', '_gps_ifd_pointer', '_segments', 'delete', 'delete_all', 'exif_version', 'get', 'get_all', 'get_file', 'get_thumbnail', 'has_exif', 'list_all']


__Далее необходимо сохранить полученные изображения без тегов в файлы. Как и в предыдущем варианте, создаем новую папку__

In [47]:
name_pth = 'no_exif_ex'
if not os.path.exists(name_pth):
    os.mkdir(name_pth)
savepth = name_pth

__Далее в цикле проходим по объектам и сохраняем их__

In [48]:
%%time
for k, v in exif_imgs.items():
    with open(os.path.join(savepth, v), 'wb') as new_img:
        new_img.write(k.get_file())

CPU times: user 11.6 ms, sys: 52.8 ms, total: 64.4 ms
Wall time: 140 ms


## Выводы

Модуль Exif работает намного быстрее,чем Pillow в части удаления тегов. При этом настройки по умолячанию в Pillow значительно изменяют объем изображения, что означает проведенные изменения поимимо удаления метаданных. Этот фактор может повлиять на качество дальнейшей обработки изображения. Поэтому дляудаления метаданных целесообразно использовать модуль EXIF. Однако для Pillow это не основной функционал. Он имеет ограмные возможности, помимо удаления тегов.